## <span style='color:#ff5f27'> 📝 Colab Users - Uncomment & Run the following 2 Cells

In [1]:
!python --version

Python 3.9.20


In [2]:
!pip uninstall hopsworks hsfs hsml -y

Found existing installation: hopsworks 4.1.0
Uninstalling hopsworks-4.1.0:
  Successfully uninstalled hopsworks-4.1.0


In [3]:
!pip install hopsworks

  Using cached hopsworks-4.1.4-py3-none-any.whl.metadata (11 kB)
Using cached hopsworks-4.1.4-py3-none-any.whl (640 kB)


In [4]:
!pip install hopsworks --quiet
!pip install xgboost==2.0.3 --quiet
!pip install scikit-learn==1.4.1.post1 --quiet
!pip install langchain==0.1.10 --quiet
!pip install bitsandbytes==0.42.0 --quiet
!pip install accelerate==0.27.2 --quiet
!pip install transformers==4.38.2 --quiet

In [5]:
!pip install wget --quiet

In [6]:
!mkdir -p functions
!cd functions && curl https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/air_quality_data_retrieval.py 
!cd functions && curl https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/context_engineering.py
!cd functions && curl https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/llm_chain.py
!cd functions && curl https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/util.py

import pandas as pd
from typing import Any, Dict, List
import datetime
import pandas as pd
import hopsworks
from hsfs.feature import Feature

def get_historical_data_for_date(date: str, feature_view, weather_fg, model) -> pd.DataFrame:
    """
    Retrieve data for a specific date from a feature view.

    Args:
        date (str): The date in the format "%Y-%m-%d".
        feature_view: The feature view object.
        model: The machine learning model used for prediction.

    Returns:
        pd.DataFrame: A DataFrame containing data for the specified date.
    """
    # Convert date string to datetime object
    date_datetime = datetime.datetime.strptime(date, "%Y-%m-%d").date()

    features_df, labels_df = feature_view.training_data(
        start_time=date_datetime,
        end_time=date_datetime + datetime.timedelta(days=1),
        # event_time=True,
        statistics_config=False
    )
    # bugfix line, shouldn't need to cast to datetime
    features_df['date'] = pd.to_date

## <span style='color:#ff5f27'> 📝 Imports

In [7]:
from xgboost import XGBRegressor
import hopsworks
from openai import OpenAI
from functions.llm_chain import (
    load_model, 
    get_llm_chain, 
    generate_response, 
    generate_response_openai,
)
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

/Users/ymm/opt/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store </span>

In [8]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
# with open('../../data/hopsworks-api-key.txt', 'r') as file:
#     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()


project = hopsworks.login()
fs = project.get_feature_store() 

2024-12-15 03:41:35,916 INFO: Initializing external client
2024-12-15 03:41:35,917 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-15 03:41:38,557 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164445


In [9]:
# Get_or_create the 'air_quality_fv' feature view
feature_view = fs.get_feature_view(
    name='air_quality_fv',
    version=4
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)

weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [26]:
# 检查可用的

Models with name 'air_quality_xgboost_model_with_lagged_data': [Model(name: 'air_quality_xgboost_model_with_lagged_data', version: 3), Model(name: 'air_quality_xgboost_model_with_lagged_data', version: 1), Model(name: 'air_quality_xgboost_model_with_lagged_data', version: 2)]


AttributeError: 'Model' object has no attribute 'list_versions'

## <span style="color:#ff5f27;">🪝 Retrieve AirQuality Model from Model Registry</span>

In [14]:
# Retrieve the model registry
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="air_quality_xgboost_model_with_lagged_data",
    version=3,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

In [13]:
import hopsworks
import os

# 重新连接
project = hopsworks.login()
mr = project.get_model_registry()

try:
    # 获取模型
    model = mr.get_model(
        name="air_quality_xgboost_model_with_lagged_data",
        version=3
    )
    
    # 尝试使用不同的路径格式
    model_base_path = model.model_path
    print(f"Base path: {model_base_path}")
    
    # 尝试不同的路径组合
    possible_paths = [
        f"{model_base_path}/3",
        f"{model_base_path}/v3",
        f"{model_base_path}_v3",
        f"{model_base_path}/version_3",
        model_base_path  # 尝试直接使用基本路径
    ]
    
    print("\nTrying different paths:")
    for path in possible_paths:
        print(f"\nAttempting path: {path}")
        try:
            local_dir = f'./model_files_{path.replace("/", "_")}'
            os.makedirs(local_dir, exist_ok=True)
            saved_model_dir = model.download(local_path=local_dir)
            print(f"Success! Model downloaded to: {saved_model_dir}")
            break
        except Exception as e:
            print(f"Failed with: {str(e)}")
            
except Exception as e:
    print(f"Main error: {str(e)}")

2024-12-15 03:44:14,875 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-15 03:44:14,899 INFO: Initializing external client
2024-12-15 03:44:14,900 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-15 03:44:16,254 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164445
Base path: /Projects/la/Models/air_quality_xgboost_model_with_lagged_data

Trying different paths:

Attempting path: /Projects/la/Models/air_quality_xgboost_model_with_lagged_data/3
Success! Model downloaded to: ./model_files__Projects_la_Models_air_quality_xgboost_model_with_lagged_data_3


In [15]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# model_air_quality = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
model_air_quality = XGBRegressor()

model_air_quality.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
model_air_quality

XGBRegressor(base_score='2.327862E1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'float',
                            'float', 'float'],
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style='color:#ff5f27'>⬇️ LLM Loading

In [12]:
import time
start_time = time.time()

# Load the LLM and its corresponding tokenizer.
model_llm, tokenizer = load_model(model_id="imiraoui/OpenHermes-2.5-Mistral-7B-sharded")

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

## <span style='color:#ff5f27'>⛓️ LangChain

In [ ]:
import time
start_time = time.time()


# Create and configure a language model chain.
llm_chain = get_llm_chain(
    model_llm,
    tokenizer,
)

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

## <span style='color:#ff5f27'>🧬 Domain-specific Evaluation Harness

**Systematic evaluations** that can run automatically in CI/CD pipelines are key to evaluating models/RAG. 


In [ ]:
QUESTION7 = "Hi!"

response7 = generate_response(
    QUESTION7,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response7)

In [ ]:
QUESTION = "Who are you?"

response = generate_response(
    QUESTION,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION1 = "What was the average air quality from 2024-01-10 till 2024-01-14?"

response1 = generate_response(
    QUESTION1, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response1)

In [ ]:
QUESTION11 = "When and what was the air quality like last week?"

response11 = generate_response(
    QUESTION11, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response11)

In [ ]:
QUESTION12 = "When and what was the minimum air quality from 2024-01-10 till 2024-01-14?"

response12 = generate_response(
    QUESTION12, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response12)

In [ ]:
QUESTION2a = "What was the air quality like last week?"

response2 = generate_response(
    QUESTION2a,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response2)

In [ ]:
QUESTION2 = "What was the air quality like yesterday?"

response2 = generate_response(
    QUESTION2,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response2)

In [ ]:
QUESTION3 = "What will the air quality be like next Tuesday?"

response3 = generate_response(
    QUESTION3, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response3)

In [ ]:
QUESTION4 = "What will the air quality be like the day after tomorrow?"

response4 = generate_response(
    QUESTION4, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response4)

In [ ]:
QUESTION5 = "What will the air quality be like this Sunday?"

response5 = generate_response(
    QUESTION5, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response5)

In [ ]:
QUESTION7 = "What will the air quality be like for the rest of the week?"

response7 = generate_response(
    QUESTION7, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response7)

In [ ]:
QUESTION = "Will the air quality be safe or not for the next week?"

response = generate_response(
    QUESTION7, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Is tomorrow's air quality level dangerous?"

response = generate_response(
    QUESTION, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Can you please explain different PM2_5 air quality levels?"

response = generate_response(
    QUESTION, 
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# !pip install openai --quiet
# !pip install gradio==3.40.1 --quiet

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
from xgboost import XGBRegressor
from functions.llm_chain import load_model, get_llm_chain, generate_response


In [ ]:
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    if y.ndim > 1 and y.shape[1] > 1:
        y = np.mean(y, axis=1)
    y /= np.max(np.abs(y))
    return transcriber({"sampling_rate": sr, "raw": y})["text"]

def generate_query_response(user_query, method, openai_api_key=None):
    if method == 'Hermes LLM':        
        response = generate_response(
            user_query,
            feature_view,
            weather_fg,
            model_air_quality,
            model_llm,
            tokenizer,
            llm_chain,
            verbose=False,
        )
        return response
    
    elif method == 'OpenAI API' and openai_api_key:
        client = OpenAI(
            api_key=openai_api_key
        )
        
        response = generate_response_openai(   
            user_query,
            feature_view,
            weather_fg,
            model_air_quality,
            client=client,
            verbose=True,
        )
        return response
        
    else:
        return "Invalid method or missing API key."

def handle_input(text_input=None, audio_input=None, method='Hermes LLM', openai_api_key=""):
    if audio_input is not None:
        user_query = transcribe(audio_input)
    else:
        user_query = text_input
    
    # Check if OpenAI API key is required but not provided
    if method == 'OpenAI API' and not openai_api_key.strip():
        return "OpenAI API key is required for this method."

    if user_query:
        return generate_query_response(user_query, method, openai_api_key)
    else:
        return "Please provide input either via text or voice."
    

# Setting up the Gradio Interface
iface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(placeholder="Type here or use voice input..."), 
        gr.Audio(), 
        gr.Radio(["Hermes LLM", "OpenAI API"], label="Choose the response generation method"),
        gr.Textbox(label="Enter your OpenAI API key (only if you selected OpenAI API):", type="password")  # Removed `optional=True`
    ],
    outputs="text",
    title="🌤️ AirQuality AI Assistant 💬",
    description="Ask your questions about air quality or use your voice to interact. Select the response generation method and provide an OpenAI API key if necessary."
)

iface.launch(share=True)


---